In [ ]:
# Importa as bibliotecas necessárias
import requests
import random
import google.generativeai as genai
import textwrap
from IPython.display import display
from IPython.display import Markdown

# Configura a API do Google Generative AI
GOOGLE_API_KEY = 'SUA_CHAVE_API_AQUI'  # Substitua pela sua chave de API
genai.configure(api_key=GOOGLE_API_KEY)

# Função para formatar a saída como Markdown
def to_markdown(text):
    text = text.replace('•', '   *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Obtém a lista de palavras do JSON
url = 'https://raw.githubusercontent.com/guilhermeonrails/api-imersao-ia/main/words.json'
resposta = requests.get(url)
data = resposta.json()

# Define os níveis do jogo
niveis = {
    1: {"pontos": 10, "premio": "Um parabéns! 🥳"},
    2: {"pontos": 20, "premio": "Um emoji maneiro! 😎"},
    3: {"pontos": 30, "premio": "Uma salva de palmas! 👏👏👏"}
}

# Inicia o jogo no nível 1
nivel_atual = 1
pontos = 0

# Inicia o chat com o Gemini
model = genai.GenerativeModel(model_name='gemini-1.0-pro')
chat = model.start_chat(history=[])

while nivel_atual <= len(niveis):
    print(f"\n--- Nível {nivel_atual} ---")

    # Escolhe uma palavra secreta aleatória
    valor_secreto = random.choice(data)
    palavra_secreta = valor_secreto['palavra']
    dica = valor_secreto['dica']

    print(f'A palavra secreta tem {len(palavra_secreta)} letras')
    print(f'A dica é -> {dica}')

    tentativas = 3
    while tentativas > 0:
        chute = input('O que você acha que é? R: ')

        if chute == palavra_secreta:
            print(f"Acertou! Você ganhou {niveis[nivel_atual]['pontos']} pontos!")
            pontos += niveis[nivel_atual]['pontos']
            print(f"Prêmio: {niveis[nivel_atual]['premio']}")
            nivel_atual += 1
            break  # Passa para o próximo nível
        else:
            tentativas -= 1
            print(f'Errou... Restam {tentativas} tentativas.')
            if tentativas > 0:
                quer_dica = input('Quer uma dica do Gemini? (s/n): ').lower()
                if quer_dica == 's':
                    prompt = f"Me dê uma dica adicional para a palavra '{palavra_secreta}', que se relaciona com '{dica}'."
                    response = chat.send_message(prompt)
                    for message in chat.history:
                        display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
                        print('-------------------------------------------')
                elif quer_dica == 'n':
                    print("Ok, continue tentando.")
                else:
                    print("Opção inválida. Tente novamente.")

    if tentativas == 0:
        print(f"Você perdeu! A palavra secreta era '{palavra_secreta}'.")
        break  # Sai do loop se perder todas as tentativas

if nivel_atual > len(niveis):
    print("\nParabéns! Você completou todos os níveis!")
print(f"\nPontuação final: {pontos}")


--- Nível 1 ---
A palavra secreta tem 3 letras
A dica é -> gerenciar e consultar bancos de dados relacionais.
O que você acha que é? R: hdj
Errou... Restam 2 tentativas.
Quer uma dica do Gemini? (s/n): n
Ok, continue tentando.
O que você acha que é? R: vc
Errou... Restam 1 tentativas.
Quer uma dica do Gemini? (s/n): n
Ok, continue tentando.
O que você acha que é? R: c
Errou... Restam 0 tentativas.
Você perdeu! A palavra secreta era 'sql'.

Pontuação final: 0
